### Постановка задачи
Построить простую модель энергопотребления здания по среднему значению,
оценить эффективность модели через метрику
\begin{equation}
RMSLE = {\sqrt{\sum_{i=1}^{n}{(log(p_i+1) - log(a_i+1))^2} \over n}}.
\end{equation}
* n - число наблюдений
* log - натуральный логарифм
* p_i - вычисленное значение метрики
* a_i - заданное значение метрики

Данные: http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8

Дополнительно сразу отсечем пустые дни и выделим час из значения времени

In [13]:
energy_0 = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.0.csv.gz")
energy_0 = energy_0[energy_0["meter_reading"] > 0]
energy_0["timestamp"] = pd.to_datetime(energy_0["timestamp"])
energy_0["hour"] = energy_0["timestamp"].dt.hour
print (energy_0.head())

      building_id  meter           timestamp  meter_reading  hour
704             0      0 2016-01-30 08:00:00        43.6839     8
725             0      0 2016-01-31 05:00:00        37.5408     5
737             0      0 2016-01-31 17:00:00        52.5571    17
2366            0      0 2016-04-08 14:00:00        59.3827    14
2923            0      0 2016-05-01 19:00:00       448.0000    19


Выделим 20% всех данных на проверку, остальные оставим на обучение

In [14]:
energy_0_train, energy_0_test = train_test_split(energy_0, test_size=0.2)
print (energy_0_train.head())

      building_id  meter           timestamp  meter_reading  hour
4825            0      0 2016-07-20 01:00:00        323.534     1
3517            0      0 2016-05-26 13:00:00        188.387    13
3559            0      0 2016-05-28 07:00:00        197.260     7
7071            0      0 2016-10-21 15:00:00        246.404    15
5880            0      0 2016-09-02 00:00:00        242.309     0


Создадим модели

Среднее и медианное значение потребление энергии по часам


In [15]:
energy_0_train_hours = energy_0_train.groupby("hour")
energy_0_train_averages = pd.DataFrame(
    {"Среднее": energy_0_train_hours.mean()["meter_reading"],
     "Медиана": energy_0_train_hours.median()["meter_reading"]})
print (energy_0_train_averages)

         Среднее   Медиана
hour                      
0     238.279586  240.2610
1     241.382585  241.9675
2     238.980453  240.9440
3     239.662556  244.3570
4     240.940122  247.0870
5     238.434567  245.7220
6     237.114315  243.6740
7     239.063956  245.7220
8     244.224576  240.2610
9     234.737587  234.1180
10    236.380806  239.5790
11    236.875381  241.6260
12    236.276650  243.6740
13    238.309390  244.6980
14    237.243223  244.3570
15    240.188601  243.6740
16    238.095350  242.9910
17    235.287743  240.9440
18    237.089515  238.8960
19    239.065248  238.2140
20    238.980502  237.8725
21    237.812913  238.8960
22    238.907500  238.2140
23    240.955200  239.5790


Функция проверки модели

𝑅𝑀𝑆𝐿𝐸=∑𝑛𝑖=1(𝑙𝑜𝑔(𝑝𝑖+1)−𝑙𝑜𝑔(𝑎𝑖+1))2𝑛⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯√.

Для вычисления метрики создадим шесть новых столбцов в тестовом наборе данных: с логарифмом значения метрики, предсказанием по среднему и по медиане, а также с квадратом разницы предсказаний и логарифма значения. Последний столбец добавим, чтобы сравнить предсказание с его отсутствием - нулями в значениях.

In [16]:
def calculate_model (x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_0_train_averages["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_0_train_averages["Медиана"][x.hour] + 1)
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = (meter_reading_log)**2
    return x

energy_0_test = energy_0_test.apply(calculate_model,
                                    axis=1, result_type="expand")
print (energy_0_test.head())

      building_id  meter           timestamp  meter_reading  hour  \
4254            0      0 2016-06-26 06:00:00        246.404     6   
5654            0      0 2016-08-23 14:00:00        290.771    14   
5460            0      0 2016-08-15 12:00:00        312.613    12   
4496            0      0 2016-07-06 08:00:00        227.975     8   
4136            0      0 2016-06-21 08:00:00        238.896     8   

      meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
4254              0.001465                0.000123             30.371370  
5654              0.041078                0.030017             32.216627  
5460              0.077804                0.061620             33.041340  
4496              0.004701                0.002732             29.524148  
4136              0.000483                0.000032             30.032652  


Теперь остается просуммировать квадраты расхождений, разделить на количество значений и извлечь квадратный корень

In [17]:
energy_0_test_median_rmsle = np.sqrt(energy_0_test["meter_reading_median_q"].sum() / len(energy_0_test))
energy_0_test_mean_rmsle = np.sqrt(energy_0_test["meter_reading_mean_q"].sum() / len(energy_0_test))
energy_0_test_zero_rmsle = np.sqrt(energy_0_test["meter_reading_zero_q"].sum() / len(energy_0_test))
print ("Качество медианы:", energy_0_test_median_rmsle)
print ("Качество среднего:", energy_0_test_mean_rmsle)
print ("Качество нуля:", energy_0_test_zero_rmsle)

Качество медианы: 0.25576939615721805
Качество среднего: 0.25316825433174944
Качество нуля: 5.441431294852722
